In [14]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [15]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# Convert pixel values to floats in the range [0, 1]
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Subtract the mean pixel value from each image
mean_pixel = x_train.mean(axis=(0,1,2), keepdims=True)
x_train -= mean_pixel
x_test -= mean_pixel

# Convert labels to one-hot encoding
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [16]:
'''
Feature Extraction is performed by ResNet50 pretrained on imagenet weights. 
Input size is 224 x 224.
'''
def feature_extractor(inputs):

  feature_extractor = tf.keras.applications.resnet.ResNet50(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
  return feature_extractor


'''
Defines final dense layers and subsequent softmax layer for classification.
'''
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)
    return x

'''
Since input image size is (32 x 32), first upsample the image by factor of (7x7) to transform it to (224 x 224)
Connect the feature extraction and "classifier" layers to build the model.
'''
def final_model(inputs):

    resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)

    resnet_feature_extractor = feature_extractor(resize)
    classification_output = classifier(resnet_feature_extractor)

    return classification_output

'''
Define the model and compile it. 
Use Stochastic Gradient Descent as the optimizer.
Use Sparse Categorical CrossEntropy as the loss function.
'''
def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(32,32,3))
  
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
  model.compile(optimizer='SGD', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  return model


model = define_compile_model()

model.summary()

2023-03-01 17:17:43.075401: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 144.0KiB (rounded to 147456)requested by op Mul
Current allocation summary follows.
2023-03-01 17:17:43.075461: I tensorflow/core/common_runtime/bfc_allocator.cc:972] BFCAllocator dump for GPU_0_bfc
2023-03-01 17:17:43.075484: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (256): 	Total Chunks: 222, Chunks in use: 222. 55.5KiB allocated for chunks. 55.5KiB in use in bin. 46.4KiB client-requested in use in bin.
2023-03-01 17:17:43.075491: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (512): 	Total Chunks: 160, Chunks in use: 160. 80.0KiB allocated for chunks. 80.0KiB in use in bin. 80.0KiB client-requested in use in bin.
2023-03-01 17:17:43.075498: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (1024): 	Total Chunks: 286, Chunks in use: 286. 287.8KiB allocated for chunks. 287.8KiB in use in bin. 286.0KiB client

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,64,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Mul]

In [11]:
resnet = build_model()
resnet.summary()

2023-03-01 15:47:44.944266: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.00MiB (rounded to 2097152)requested by op Add
Current allocation summary follows.
2023-03-01 15:47:44.944325: I tensorflow/core/common_runtime/bfc_allocator.cc:972] BFCAllocator dump for GPU_0_bfc
2023-03-01 15:47:44.944347: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (256): 	Total Chunks: 175, Chunks in use: 175. 43.8KiB allocated for chunks. 43.8KiB in use in bin. 35.2KiB client-requested in use in bin.
2023-03-01 15:47:44.944354: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (512): 	Total Chunks: 160, Chunks in use: 160. 80.0KiB allocated for chunks. 80.0KiB in use in bin. 80.0KiB client-requested in use in bin.
2023-03-01 15:47:44.944361: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (1024): 	Total Chunks: 261, Chunks in use: 261. 262.8KiB allocated for chunks. 262.8KiB in use in bin. 261.0KiB client

ResourceExhaustedError: OOM when allocating tensor with shape[1,1,512,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add]

In [6]:
resnet = build_model(x_train.shape[1:], num_classes)
resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.1)

2023-03-01 15:35:04.958301: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 38.18M (40032512 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-03-01 15:35:04.959062: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 38.18M (40032512 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-03-01 15:35:14.960142: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 38.18M (40032512 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-03-01 15:35:14.961115: I tensorflow/stream_executor/cuda/cuda_driver.cc:789] failed to allocate 38.18M (40032512 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-03-01 15:35:14.961141: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 9.00MiB (rounded to 9437184)requested by op Mul
Current allocation summary follows.
2023-03-01 15:35:14.961163: I tensorflow/core/common_runtime/b

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Mul]

In [ ]:
plt.plot(resnet.history['accuracy'], label='accuracy')
plt.plot(resnet.history['val_accuracy'], label = 'val_accuracy')
plt.title('ResNet')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)